In [ ]:
# !sudo pip install transformers peft evaluate seqeval

In [ ]:
import numpy as np
import pandas as pd

ANNOT_DIR = "/resources/data/restricted/anonymization"

train = pd.read_csv(
    f"{ANNOT_DIR}/train.txt",
    sep=" ",
    names=["token", "label"],
    skip_blank_lines=False,
)

val = pd.read_csv(
    f"{ANNOT_DIR}/dev.txt",
    sep=" ",
    names=["token", "label"],
    skip_blank_lines=False,
)

test = pd.read_csv(
    f"{ANNOT_DIR}/test.txt",
    sep=" ",
    names=["token", "label"],
    skip_blank_lines=False,
)


cat_count = dict(train["label"].value_counts())
categories = {v: k for k, v in enumerate(train["label"].unique()) if not pd.isna(v)}
label2code = {k: i for i, (k, v) in enumerate(categories.items())}
code2label = {v: k for k, v in label2code.items()}

print("train:", len(train))
print("val:", len(val))
print("test:", len(test))
print("nlabels:", len(code2label))

In [ ]:
code2label

In [ ]:
label2code

In [ ]:
import numpy as np
from more_itertools import unzip
from tqdm.auto import tqdm
from more_itertools import split_at


def get_hg_format(df):
    df = df.copy()

    df["tags"] = df["label"].map(lambda x: label2code.get(x, label2code["O"]))
    items = []
    indices, rows = unzip(df.iterrows())
    tuples = map(lambda row: (row["token"], row["tags"]), rows)
    tuples = split_at(tuples, lambda x: pd.isna(x[0]))

    for i, paragraph in enumerate(tuples):
        df = pd.DataFrame(paragraph)
        # if not 3 <= len(df) <= 10:
        #     continue
        if not len(df):
            continue
        tokens = df[0].values
        labels = df[1].values.astype(int)
        nlabels = (len(labels[labels > 0]),)

        if any(np.isnan(labels)):
            continue

        # if nlabels == 0 and np.random.rand() < 0.99:
        # print('skipping no entity paragraph')
        # continue

        if len(tokens) != len(labels):
            print("mismatch size")
            continue

        items.append(
            {
                "n_labels": nlabels,
                "tokens": list(tokens),
                "tags": labels,
            }
        )

    return items

In [ ]:
train["token"]

In [ ]:
asd = get_hg_format(train)

In [ ]:
len(asd)

In [ ]:
from datasets import DatasetDict, Dataset

dataset = DatasetDict(
    {
        "train": Dataset.from_list(get_hg_format(train)),
        "validation": Dataset.from_list(get_hg_format(val)),
        "test": Dataset.from_list(get_hg_format(test)),
    }
)
dataset

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
)
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
)
import evaluate
import torch
import numpy as np

# model_checkpoint = "roberta-large"
model_checkpoint = "dccuchile/bert-base-spanish-wwm-cased"
lr = 1e-3
batch_size = 8
num_epochs = 10

In [ ]:
seqeval = evaluate.load("seqeval")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
len(tokenized_dataset["train"])

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
dataset["train"][10]

In [ ]:
len(label2code), len(code2label)

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label2code.keys()),
    id2label=code2label,
    label2id=label2code,
)

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS,
    inference_mode=False,
    r=16,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="all",
)

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
training_args = TrainingArguments(
    output_dir="beto-lora-aymurai-ner",
    learning_rate=lr,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    # predict_with_generate=True,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_steps=100,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
val = tokenized_dataset["validation"].select(range(1000))
val

In [ ]:
import seqeval

seqeval = evaluate.load("seqeval")


def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [code2label.get(p) for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [code2label.get(l, "O") for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"].select(range(600)),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [ ]:
trainer.evaluate(tokenized_dataset["validation"].select(range(1000)))

In [ ]:
trainer.train()

In [ ]:
MODEL_PATH = "./beto-lora-aymurai-ner/model/"

trainer.model.save_pretrained()

In [ ]:
from peft import PeftConfig, PeftModelForTokenClassification, LoraConfig, TaskType, get_peft_model
from transformers import AutoTokenizer, AutoModelForTokenClassification


MODEL_PATH = "./beto-lora-aymurai-ner/model"

# Load peft config for pre-trained checkpoint etc.
peft_config = PeftConfig.from_pretrained(MODEL_PATH)
lora_config = LoraConfig(
    base_model_name_or_path=model_checkpoint,
    task_type=TaskType.TOKEN_CLS,
    inference_mode=True,
    r=16,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="all",
)

In [ ]:
peft_config

In [ ]:
lora_config

In [ ]:
# load base LLM model and tokenizer
model = AutoModelForTokenClassification.from_pretrained(
    peft_config.base_model_name_or_path,
    num_labels=len(label2code.keys()),
    id2label=code2label,
    label2id=label2code,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)

In [ ]:
# model = PeftModelForTokenClassification(model, peft_config)
model = get_peft_model(model, peft_config)

In [ ]:
model.eval()

In [ ]:
model.print_trainable_parameters()

In [ ]:
model.config.id2label

In [ ]:
model.get_

In [ ]:
text = "El imputado Ramiro Ramallo Martinez dni 88.384.425"
inputs = tokenizer(text, return_tensors="pt")
model.to("cpu")
with torch.no_grad():
    logits = model(**inputs).logits

tokens = inputs.tokens()

predictions = torch.argmax(logits, dim=2)

for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))

In [ ]:
dataset

In [ ]:
trainer.evaluate(tokenized_dataset["test"].select(range(1000)))